### With predicted rental prices, we are now interested in the growth rates for each SA2

In [2]:
import pandas as pd
import glob
import os

#path = r'../data/curated/2023_2027_rental_prediction_xgboost/' # use your path
path = r'../data/curated/random_forest_pred/'
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
predicted = merged_df.groupby(['year', 'sa2_2021'], as_index=False).agg({'predicted_price': 'mean'})
predicted

,year,sa2_2021,predicted_price
0,2023,201011008,322.956986
1,2023,201011488,302.071106
2,2023,201021008,325.702252
3,2023,201031008,281.075497
4,2023,201031024,296.889565
...,...,...,...
675,2027,216031424,301.774784
676,2027,216031600,297.825722
677,2027,217011424,270.428197
678,2027,217031472,293.288460


In [3]:
rental_2022 = pd.read_csv('../data/curated/merged_dataset/2022_merged_data.csv')
rental_2022.dropna(inplace=True)
rental_2022 = rental_2022.groupby(['year', 'sa2_2021'], as_index=False).agg({'weekly_rent': 'mean'})
rental_2022.rename({'weekly_rent': 'predicted_price'}, axis=1, inplace=True)
rental_2022
#rental_2022[rental_2022['sa2_2021']==206041127]
#set(rental_2022['sa2_2021'].unique()) - set(predicted['sa2_2021'].unique())

,year,sa2_2021,predicted_price
0,2022,201011001,437.468013
1,2022,201011002,421.270053
2,2022,201011005,402.559524
3,2022,201011006,440.000000
4,2022,201011007,530.625000
...,...,...,...
494,2022,217031476,518.750000
495,2022,217041477,414.069767
496,2022,217041478,771.301370
497,2022,217041479,437.025641


In [4]:
growth_df = pd.concat([rental_2022, predicted], axis=0).reset_index(drop=True)
#df[df['sa2_2021']==201011001]
growth_df

,year,sa2_2021,predicted_price
0,2022,201011001,437.468013
1,2022,201011002,421.270053
2,2022,201011005,402.559524
3,2022,201011006,440.000000
4,2022,201011007,530.625000
...,...,...,...
1174,2027,216031424,301.774784
1175,2027,216031600,297.825722
1176,2027,217011424,270.428197
1177,2027,217031472,293.288460


In [5]:
subset = growth_df[growth_df['sa2_2021']==201011001]
subset
growth_dict = dict()
for sa2 in growth_df['sa2_2021'].unique():
    growth_dict[sa2] = []
    subset = growth_df[growth_df['sa2_2021']==sa2]
    for i in range(len(subset['predicted_price'])-1):
        # (current year price - past year price) / past year price
        growth_rate = (subset.iloc[i+1]['predicted_price'] - subset.iloc[i]['predicted_price'])/subset.iloc[i]['predicted_price']
        growth_dict[sa2].append(growth_rate)


In [6]:
growth_dict

{201011001: [],
 201011002: [],
 201011005: [],
 201011006: [],
 201011007: [],
 201011008: [-0.1194656528352023,
  0.006321407519018682,
  0.002860273789458275,
  0.0019728890456222684,
  0.0009652687713379873],
 201011481: [],
 201011482: [],
 201011483: [],
 201011484: [],
 201021009: [],
 201021010: [],
 201021011: [],
 201021012: [],
 201031013: [],
 201031014: [],
 201031015: [],
 201031016: [],
 201031017: [],
 202011018: [],
 202011019: [],
 202011020: [],
 202011021: [],
 202011022: [],
 202011023: [],
 202011024: [-0.37386238319877907,
  0.0007266399117907991,
  0.0013441811352868484,
  0.0016310161814316717,
  0.002736861859515438],
 202011025: [],
 202021026: [],
 202021027: [],
 202021028: [],
 202021029: [],
 202021030: [],
 202021031: [],
 202031032: [],
 202031033: [],
 203011034: [],
 203011035: [],
 203011036: [],
 203021037: [],
 203021039: [],
 203021040: [-0.09199236395204702,
  0.02612861359637628,
  0.00723361969608449,
  0.02157472388566443,
  0.0097419838097074

In [7]:
import numpy as np
for key in growth_dict.keys():
    growth_dict[key] = np.mean(growth_dict[key])
growth_dict

/home/hyunjinp/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyunjinp/.local/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{201011001: nan,
 201011002: nan,
 201011005: nan,
 201011006: nan,
 201011007: nan,
 201011008: -0.02146916274195302,
 201011481: nan,
 201011482: nan,
 201011483: nan,
 201011484: nan,
 201021009: nan,
 201021010: nan,
 201021011: nan,
 201021012: nan,
 201031013: nan,
 201031014: nan,
 201031015: nan,
 201031016: nan,
 201031017: nan,
 202011018: nan,
 202011019: nan,
 202011020: nan,
 202011021: nan,
 202011022: nan,
 202011023: nan,
 202011024: -0.07348473682215087,
 202011025: nan,
 202021026: nan,
 202021027: nan,
 202021028: nan,
 202021029: nan,
 202021030: nan,
 202021031: nan,
 202031032: nan,
 202031033: nan,
 203011034: nan,
 203011035: nan,
 203011036: nan,
 203021037: nan,
 203021039: nan,
 203021040: -0.005462684592842869,
 203021042: nan,
 203021043: nan,
 203021044: nan,
 203021045: nan,
 203021046: nan,
 203021047: nan,
 203021485: nan,
 203021486: nan,
 203021487: nan,
 203021488: 0.006644496427269135,
 203031048: nan,
 203031049: nan,
 203031051: nan,
 203031052: n